In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
from underwriting import client_surveillance_jupyter

In [4]:
COMPANY_IDENTIFIER = ['WVN']
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

In [5]:
#fetch download report and lisences
company_licenses_query = create_queries.create_company_licenses_query(COMPANY_IDENTIFIER)
company_download_summaries_query = create_queries.create_company_download_summaries_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)

company_licenses_dataframe = pd.read_sql_query(company_licenses_query, engine)
company_download_summaries_dataframe = pd.read_sql_query(company_download_summaries_query, engine)

In [6]:
company_licenses_dataframe

,us_state,license_number,license_category,legal_name,is_current,license_status,rollup_id,license_description,company_id,facility_row_id
0,CA,C11-0000371-LIC,Distributor,IRRATIONAL RACCOON LLC,True,Active,CA-01956,Distributor,ce697634-40bc-4b7d-a8db-d0e26efe7a0b,None
1,CA,C11-0000497-LIC,Distributor,ZASP,True,Active,CA-04092,Distributor,ce697634-40bc-4b7d-a8db-d0e26efe7a0b,None


## Check download status summaries

In [7]:
license_numbers = company_download_summaries_dataframe['license_number'].unique()
download_summary_records = company_download_summaries_dataframe.to_dict('records')

In [8]:
license_numbers

array(['C11-0000371-LIC', 'C11-0000497-LIC'], dtype=object)

In [9]:
license_numbers = list(license_numbers)

In [10]:
license_numbers

['C11-0000371-LIC', 'C11-0000497-LIC']

In [11]:
bad_download_history = client_surveillance_jupyter.check_company_license_download(license_numbers,download_summary_records)
bad_download_history

Verifying download summaries for license C11-0000371-LIC...
Earliest download summary: 2020-01-01
Latest download summary: 2022-07-05

Verifying download summaries for license C11-0000497-LIC...
Earliest download summary: 2020-01-01
Latest download summary: 2022-07-05

[SUCCESS] All download summaries look good!


defaultdict(list, {})

# License check

In [12]:
company_licenses_dataframe

,us_state,license_number,license_category,legal_name,is_current,license_status,rollup_id,license_description,company_id,facility_row_id
0,CA,C11-0000371-LIC,Distributor,IRRATIONAL RACCOON LLC,True,Active,CA-01956,Distributor,ce697634-40bc-4b7d-a8db-d0e26efe7a0b,None
1,CA,C11-0000497-LIC,Distributor,ZASP,True,Active,CA-04092,Distributor,ce697634-40bc-4b7d-a8db-d0e26efe7a0b,None


In [13]:
company_state = list(company_licenses_dataframe['us_state'].unique())[0]
company_state

'CA'

## Pull data

In [14]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=license_numbers,
)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=license_numbers,
)
company_unknown_transfer_packages_query = create_queries.create_company_unknown_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
#     license_numbers=license_numbers,
)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(
    COMPANY_IDENTIFIER,
    SALES_TRANSACTIONS_START_DATE,
    license_numbers=license_numbers,
)
company_sales_receipts_query = create_queries.create_company_sales_receipts_query(
    COMPANY_IDENTIFIER,
    SALES_TRANSACTIONS_START_DATE,
    license_numbers=license_numbers,
)
company_sales_receipts_with_transactions_query = create_queries.create_company_sales_receipts_with_transactions_query(
    COMPANY_IDENTIFIER,
    SALES_TRANSACTIONS_START_DATE,
    license_numbers=license_numbers,
)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER,
    include_quantity_zero=True,
    license_numbers=license_numbers,
)



company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pd.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_unknown_transfer_packages_dataframe = pd.read_sql_query(company_unknown_transfer_packages_query, engine)
company_sales_transactions_dataframe = pd.read_sql_query(company_sales_transactions_query, engine)
company_sales_receipts_dataframe = pd.read_sql_query(company_sales_receipts_query, engine)
company_sales_receipts_with_transactions_dataframe = pd.read_sql_query(company_sales_receipts_with_transactions_query, engine)
company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)

In [15]:
unknown_package_count = client_surveillance_jupyter.check_unknown_transfer_packages(company_unknown_transfer_packages_dataframe)
unknown_package_count

[SUCCESS] No unknown transfer packages!


0

In [16]:
#make sure we did pulled all location's data
company_incoming_transfer_packages_dataframe['license_number'].unique()

array(['C11-0000371-LIC', 'C11-0000497-LIC'], dtype=object)

In [17]:
company_outgoing_transfer_packages_dataframe['license_number'].unique()

array(['C11-0000371-LIC', 'C11-0000497-LIC'], dtype=object)

In [19]:
df_in = company_incoming_transfer_packages_dataframe
df_out = company_outgoing_transfer_packages_dataframe
df_in['created_date'] = pd.to_datetime(df_in['created_date'])
df_out['created_date'] = pd.to_datetime(df_out['created_date'])


In [20]:
df_in_2021 = df_in[df_in['created_date'] >= '2021-01-01']
df_out_2021 = df_out[df_out['created_date'] >= '2021-01-01']

In [23]:
df_in_int = df_in[(df_in['delivery_type'] == 'INCOMING_INTERNAL')]
df_in_ext = df_in[(df_in['delivery_type'] != 'INCOMING_INTERNAL')]

In [25]:
p_out_2021 = set(df_out_2021['package_id'].unique())

In [26]:
p_in = set(df_in['package_id'].unique())

In [33]:
len(p_out_2021-p_in)

87885

In [37]:
df_in_ext['product_name_lower'] = df_in_ext['product_name'].str.lower()

1. not a sample product
2. not trading or testing samples
3.shipped uom == received uom

In [104]:
df_in_ext_clean = df_in_ext[(~(df_in_ext['product_name_lower'].str.contains('sample')))&(~df_in_ext['is_testing_sample'])\
                           &(~df_in_ext['is_trade_sample']) &(~df_in_ext['received_datetime'].isnull()) &\
                           (df_in_ext['shipped_unit_of_measure'] == df_in_ext['received_unit_of_measure'])]





In [105]:
df_in_ext_clean.shape

(21687, 32)

In [176]:
df_out[df_out['recipient_facility_license_number'] == 'C9-0000146-LIC'][['manifest_number','receiver_wholesale_price']].groupby('manifest_number').sum()


,receiver_wholesale_price
manifest_number,
0000228152,1152.00
0000258158,900.00
0000273650,2.00
0000341980,960.00
0000354629,2066.04
0000401882,1280.05
0000450108,2400.00
0000463841,1954.00
0000464686,500.00


In [252]:
# look up a given PO
a = df_out[df_out['manifest_number'] == '0002756680']
a

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name
30716,OUTGOING_TO_PAYOR,C11-0000371-LIC,0002756680,2021-11-25,2021-11-30 23:45:47+00:00,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C10-0000695-LIC,EMBARC TAHOE LLC,Wholesale Manifest,Wholesale,19998471,1A4060300005F51000041805,transfer,1A4060300005F51000037528,CHMDRG1082521,Accepted,False,False,Flower (packaged eighth - each),PPK-ChemDriver-H-F-3.5,passed,1920.0,96.0,Each,96.0,Each,1920.0,3.5,Grams
30726,OUTGOING_TO_PAYOR,C11-0000371-LIC,0002756680,2021-11-25,2021-11-30 23:45:47+00:00,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C10-0000695-LIC,EMBARC TAHOE LLC,Wholesale Manifest,Wholesale,19998472,1A4060300005F51000041806,transfer,1A4060300005F51000037530,CHMRESG1082521,Accepted,False,False,Flower (packaged eighth - each),PPK-ChemReserve-H-F-3.5,passed,1280.0,64.0,Each,64.0,Each,1280.0,3.5,Grams
30761,OUTGOING_TO_PAYOR,C11-0000371-LIC,0002756680,2021-11-25,2021-11-30 23:45:47+00:00,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C10-0000695-LIC,EMBARC TAHOE LLC,Wholesale Manifest,Wholesale,19998473,1A4060300005F51000041807,transfer,1A4060300005F51000030182,Peanut Butter Breath #6-04-11-2021-0006,Accepted,False,False,Flower (packaged eighth - each),PPK-PBB-H-F-3.5,passed,1152.0,64.0,Each,64.0,Each,1152.0,3.5,Grams


In [254]:
#make sure we cannot find the package in incoming data
df_in[df_in['package_id'].isin(list(a['package_id'].unique()))]

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipment_transaction_type,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type_1,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,receiver_wholesale_price,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name


In [247]:
# find out where wovn get the products from the above PO
df_in[df_in['product_name'] == 'PPK-PBB-H-F-3.5']

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipment_transaction_type,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type_1,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,receiver_wholesale_price,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
747,INCOMING_INTERNAL,C11-0000371-LIC,0003517157,2022-05-20,2022-05-25 18:24:04+00:00,None,C11-0000497-LIC,ZASP,C11-0000371-LIC,IRRATIONAL RACCOON LLC,Transfer,Standard,25358714,1A4060300008981000180194,transfer,1A4060300008981000181699,Peanut Butter Breath #6-08-24-2021-0009,Accepted,False,False,Flower (packaged eighth - each),PPK-PBB-H-F-3.5,passed,NaN,512.0,Each,NaN,512.0,Each,3.5,Grams
777,INCOMING_FROM_VENDOR,C11-0000497-LIC,0003509989,2022-05-19,2022-05-24 17:55:03+00:00,Standard,C11-0000292-LIC,"Pure CA, LLC",C11-0000497-LIC,ZASP,Transfer,Standard,24792907,1A4060300008981000179774,transfer,1A4060300005F51000045999,Peanut Butter Breath #6-08-24-2021-0009,Accepted,False,False,Flower (packaged eighth - each),PPK-PBB-H-F-3.5,passed,NaN,223.0,Each,NaN,223.0,Each,3.5,Grams
1443,INCOMING_INTERNAL,C11-0000497-LIC,0003250839,2022-03-23,2022-03-28 22:37:39+00:00,None,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,23480476,1A4060300005F51000052913,transfer,1A4060300005F51000043236,Peanut Butter Breath #6-08-24-2021-0009,Accepted,False,False,Flower (packaged eighth - each),PPK-PBB-H-F-3.5,passed,NaN,480.0,Each,NaN,480.0,Each,3.5,Grams
3418,INCOMING_INTERNAL,C11-0000497-LIC,0002794722,2021-12-06,2021-12-07 19:44:53+00:00,Standard,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,20239228,1A4060300005F51000042544,transfer,1A4060300005F51000030182,Peanut Butter Breath #6-04-11-2021-0006,Accepted,False,False,Flower (packaged eighth - each),PPK-PBB-H-F-3.5,passed,18.0,256.0,Each,18.0,256.0,Each,3.5,Grams
4162,INCOMING_INTERNAL,C11-0000371-LIC,0002607548,2021-10-22,2021-10-26 16:30:02+00:00,Standard,C11-0000497-LIC,ZASP,C11-0000371-LIC,IRRATIONAL RACCOON LLC,Transfer,Standard,18965657,1A4060300008981000162939,transfer,1A4060300005F51000038186,Peanut Butter Breath #6-04-11-2021-0006,Accepted,False,False,Flower (packaged eighth - each),PPK-PBB-H-F-3.5,passed,NaN,512.0,Each,NaN,512.0,Each,3.5,Grams
4267,INCOMING_INTERNAL,C11-0000497-LIC,0002555321,2021-10-10,2021-10-12 21:17:57+00:00,Standard,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,18600655,1A4060300005F51000038186,transfer,1A4060300005F51000037536,Peanut Butter Breath #6-04-11-2021-0006,Accepted,False,False,Flower (packaged eighth - each),PPK-PBB-H-F-3.5,passed,978.0,960.0,Each,978.0,960.0,Each,3.5,Grams


In [234]:
df_in[df_in['manifest_number'] == '0002555321']

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipment_transaction_type,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type_1,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,receiver_wholesale_price,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
4263,INCOMING_INTERNAL,C11-0000497-LIC,0002555321,2021-10-10,2021-10-12 21:17:57+00:00,Standard,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,18600664,1A4060300005F51000038193,transfer,1A4060300005F51000037530,CHMRESG1082521,Accepted,False,False,Flower (packaged eighth - each),PPK-ChemReserve-H-F-3.5,passed,20.0,1600.0,Each,20.0,1600.0,Each,3.5,Grams
4264,INCOMING_INTERNAL,C11-0000497-LIC,0002555321,2021-10-10,2021-10-12 21:17:57+00:00,Standard,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,18600658,1A4060300005F51000038187,transfer,1A4060300005F51000037537,Peanut Butter Breath #6-04-11-2021-0006,Accepted,False,False,Flower (packaged eighth - each),PPK-PBB-H-F-3.5-MOQ1,passed,18.0,8.0,Each,18.0,8.0,Each,3.5,Grams
4265,INCOMING_INTERNAL,C11-0000497-LIC,0002555321,2021-10-10,2021-10-12 21:17:57+00:00,Standard,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,18600656,1A4060300005F51000038188,transfer,1A4060300005F51000037535,Modified Mintz-07-05-2021-0106,Accepted,False,False,Flower (packaged eighth - each),PPK-ModifiedMintz-I-F-3.5-MOQ1,passed,18.0,8.0,Each,18.0,8.0,Each,3.5,Grams
4266,INCOMING_INTERNAL,C11-0000497-LIC,0002555321,2021-10-10,2021-10-12 21:17:57+00:00,Standard,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,18600660,1A4060300005F51000038185,transfer,1A4060300005F51000037532,S208 - GAC - 3715,Accepted,False,False,Flower (packaged eighth - each),PPK-GNC-I-F-3.5,passed,18.0,960.0,Each,18.0,960.0,Each,3.5,Grams
4267,INCOMING_INTERNAL,C11-0000497-LIC,0002555321,2021-10-10,2021-10-12 21:17:57+00:00,Standard,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,18600655,1A4060300005F51000038186,transfer,1A4060300005F51000037536,Peanut Butter Breath #6-04-11-2021-0006,Accepted,False,False,Flower (packaged eighth - each),PPK-PBB-H-F-3.5,passed,978.0,960.0,Each,978.0,960.0,Each,3.5,Grams
4268,INCOMING_INTERNAL,C11-0000497-LIC,0002555321,2021-10-10,2021-10-12 21:17:57+00:00,Standard,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,18600666,1A4060300005F51000038191,transfer,1A4060300005F51000037528,CHMDRG1082521,Accepted,False,False,Flower (packaged eighth - each),PPK-ChemDriver-H-F-3.5,passed,20.0,1600.0,Each,20.0,1600.0,Each,3.5,Grams
4269,INCOMING_INTERNAL,C11-0000497-LIC,0002555321,2021-10-10,2021-10-12 21:17:57+00:00,Standard,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,18600665,1A4060300005F51000038190,transfer,1A4060300005F51000037529,CHMDRG1082521,Accepted,False,False,Flower (packaged eighth - each),PPK-ChemDriver-H-F-3.5-MOQ1,passed,20.0,16.0,Each,20.0,16.0,Each,3.5,Grams
4270,INCOMING_INTERNAL,C11-0000497-LIC,0002555321,2021-10-10,2021-10-12 21:17:57+00:00,Standard,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,18600662,1A4060300005F51000038183,transfer,1A4060300005F51000037526,S215 - BDR - 3716,Accepted,False,False,Flower (packaged eighth - each),PPK-BlueberryHaze-S-F-3.5,passed,18.0,384.0,Each,18.0,384.0,Each,3.5,Grams
4271,INCOMING_INTERNAL,C11-0000497-LIC,0002555321,2021-10-10,2021-10-12 21:17:57+00:00,Standard,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,18600663,1A4060300005F51000038192,transfer,1A4060300005F51000

In [237]:
df_out[df_out['product_name'] == 'PPK-ChemDriver-H-F-3.5']

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name
313,OUTGOING_TO_PAYOR,C11-0000497-LIC,0003700621,2022-06-30,NaT,C11-0000497-LIC,ZASP,C11-0001076-LIC,"URBAN BUDS, LLC",Wholesale Manifest,Wholesale,26600141,1A4060300008981000183749,transfer,1A4060300008981000179779,CHMDRG1082521,Shipped,False,False,Flower (packaged eighth - each),PPK-ChemDriver-H-F-3.5,passed,384.00,192.0,Each,NaN,None,NaN,3.5,Grams
2513,OUTGOING_INTERNAL,C11-0000371-LIC,0003629508,2022-06-15,2022-06-17 19:38:52+00:00,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,26110982,1A4060300005F51000060404,transfer,1A4060300005F51000037528,CHMDRG1082521,Accepted,False,False,Flower (packaged eighth - each),PPK-ChemDriver-H-F-3.5,passed,1268.00,1248.0,Each,1248.0,Each,1268.00,3.5,Grams
2588,OUTGOING_INTERNAL,C11-0000371-LIC,0003629508,2022-06-15,2022-06-17 19:38:52+00:00,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000497-LIC,ZASP,Transfer,Standard,26110983,1A4060300005F51000060405,transfer,1A4060300005F51000058821,CHMDRG1082521,Accepted,False,False,Flower (packaged eighth - each),PPK-ChemDriver-H-F-3.5,passed,20.00,320.0,Each,320.0,Each,20.00,3.5,Grams
2976,OUTGOING_TO_PAYOR,C11-0000371-LIC,0003608231,2022-06-10,2022-06-13 19:47:06+00:00,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000353-LIC,"PACAFI, INC.",Transfer,Standard,25968475,1A4060300005F51000059971,transfer,1A4060300005F51000037528,CHMDRG1082521,Accepted,False,False,Flower (packaged eighth - each),PPK-ChemDriver-H-F-3.5,passed,NaN,128.0,Each,128.0,Each,NaN,3.5,Grams
3179,OUTGOING_TO_PAYOR,C11-0000371-LIC,0003607672,2022-06-10,2022-06-13 21:56:23+00:00,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000353-LIC,"PACAFI, INC.",Transfer,Standard,25965519,1A4060300005F51000060002,transfer,1A4060300005F51000037528,CHMDRG1082521,Accepted,False,False,Flower (packaged eighth - each),PPK-ChemDriver-H-F-3.5,passed,NaN,64.0,Each,64.0,Each,NaN,3.5,Grams
3181,OUTGOING_TO_PAYOR,C11-0000371-LIC,0003607638,2022-06-10,2022-06-13 19:48:16+00:00,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000353-LIC,"PACAFI, INC.",Transfer,Standard,25964249,1A4060300005F51000059985,transfer,1A4060300005F51000037528,CHMDRG1082521,Accepted,False,False,Flower (packaged eighth - each),PPK-ChemDriver-H-F-3.5,passed,NaN,128.0,Each,128.0,Each,NaN,3.5,Grams
3193,OUTGOING_TO_PAYOR,C11-0000371-LIC,0003607557,2022-06-10,2022-06-13 20:35:29+00:00,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000353-LIC,"PACAFI, INC.",Transfer,Standard,25963866,1A4060300005F51000060055,transfer,1A4060300005F51000037528,CHMDRG1082521,Accepted,False,False,Flower (packaged eighth - each),PPK-ChemDriver-H-F-3.5,passed,384.00,128.0,Each,128.0,Each,384.00,3.5,Grams
3223,OUTGOING_TO_PAYOR,C11-0000371-LIC,0003607862,2022-06-10,2022-06-13 19:49:06+00:00,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000353-LIC,"PACAFI, INC.",Transfer,Standard,25967347,1A4060300005F51000060032,transfer,1A4060300005F51000037528,CHMDRG1082521,Accepted,False,False,Flower (packaged eighth - each),PPK-ChemDriver-H-F-3.5,passed,NaN,128.0,Each,128.0,Each,NaN,3.5,Grams
3326,OUTGOING_TO_PAYOR,C11-0000371-LIC,0003607746,2022-06-10,2022-06-13 21:02:01+00:00,C11-0000371-LIC,IRRATIONAL RACCOON LLC,C11-0000353-LIC,"PACAFI, INC.",Transfer,Standard,25965912,1A4060300005F51000060024,transfer,1A4060300005F51000037528,CHMDRG1082521,Accepted,False,False,Flower (packaged eighth - each),PPK-ChemDriver-H-F-3.5,passed,NaN,128.0,Each,128.0,Each,NaN,3.5,G

In [245]:
df_in[df_in['package_id'] == '18520077']

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipment_transaction_type,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type_1,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,receiver_wholesale_price,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name


# test

In [128]:
t1 = set(df_in_ext_clean[df_in_ext_clean['received_datetime'] >= '2022-01-01']['package_id'].unique())

In [168]:
len(t1)

1804

In [130]:
t2 = set(df_out['package_id'].unique())

In [152]:
s = list(t1.intersection(t2))

In [169]:
len(s)

115

In [163]:
df_in[df_in['package_id'].isin (s)]

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipment_transaction_type,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type_1,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,receiver_wholesale_price,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
153,INCOMING_FROM_VENDOR,C11-0000497-LIC,0003676726,2022-06-27,2022-07-01 16:01:36+00:00,Standard,C10-0001014-LIC,THIRD & KENMORE LLC,C11-0000497-LIC,ZASP,Return,Standard,25131220,1A4060300008981000181595,transfer,1A4060300008981000179945,".2020-10-30_Kirkwood OG_P4., .2020-10-31_Kirkw...",Accepted,False,False,Edible (weight - each),FLAV-Blueberry-Belts-M-E-100,passed,NaN,27.0,Each,NaN,27.0,Each,40.0,Grams
156,INCOMING_FROM_VENDOR,C11-0000497-LIC,0003676726,2022-06-27,2022-07-01 16:01:36+00:00,Standard,C10-0001014-LIC,THIRD & KENMORE LLC,C11-0000497-LIC,ZASP,Return,Standard,25131230,1A4060300008981000181605,transfer,1A4060300008981000179938,"10-16 BBA (BR#2) Harvest, 10-20 Biscotti Entir...",Accepted,False,False,Edible (weight - each),FLAV-Watermelon-Rings-M-E-100,passed,NaN,28.0,Each,NaN,28.0,Each,40.0,Grams
162,INCOMING_FROM_VENDOR,C11-0000497-LIC,0003676726,2022-06-27,2022-07-01 16:01:36+00:00,Standard,C10-0001014-LIC,THIRD & KENMORE LLC,C11-0000497-LIC,ZASP,Return,Standard,25131228,1A4060300008981000181603,transfer,1A4060300008981000165155,"#19, #20, #21, #22, #24, #25, #26, #27 / 8.10....",Accepted,False,False,Edible (weight - each),FLAV-RootBeer-Drops-M-E-100,passed,NaN,20.0,Each,NaN,20.0,Each,40.0,Grams
163,INCOMING_FROM_VENDOR,C11-0000497-LIC,0003676726,2022-06-27,2022-07-01 16:01:36+00:00,Standard,C10-0001014-LIC,THIRD & KENMORE LLC,C11-0000497-LIC,ZASP,Return,Standard,25131246,1A4060300008981000181619,transfer,1A4060300008981000175804,"10/14/2021 Lota harvest Bed Bed 43, 10/14/2021...",Accepted,False,False,Vape Cartridge (weight - each),FLAV-Biscotti-I-DV-1.0,passed,NaN,27.0,Each,NaN,27.0,Each,1.0,Grams
167,INCOMING_FROM_VENDOR,C11-0000497-LIC,0003676726,2022-06-27,2022-07-01 16:01:36+00:00,Standard,C10-0001014-LIC,THIRD & KENMORE LLC,C11-0000497-LIC,ZASP,Return,Standard,25131227,1A4060300008981000181602,transfer,1A4060300008981000176687,"#19, #20, #21, #22, #24, #25, #26, #55, #56, #...",Accepted,False,False,Edible (weight - each),FLAV-Lemon-Drops-M-E-100-MOQ30,passed,NaN,29.0,Each,NaN,29.0,Each,49.0,Grams
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2633,INCOMING_FROM_VENDOR,C11-0000497-LIC,0002949880,2022-01-13,2022-01-13 22:02:02+00:00,Standard,C10-0000223-LIC,L B GREEN ROOM,C11-0000497-LIC,ZASP,Return,Standard,21188225,1A4060300008981000168709,transfer,1A4060300008981000032236,".2020-10-30_Kirkwood OG_P4., .2020-10-31_Kirkw...",Accepted,False,False,Edible (weight - each),FLAV-Mango-Belts-M-E-100-MOQ1,passed,NaN,40.0,Each,NaN,40.0,Each,40.0,Grams
2635,INCOMING_FROM_VENDOR,C11-0000497-LIC,0002949880,2022-01-13,2022-01-13 22:02:02+00:00,Standard,C10-0000223-LIC,L B GREEN ROOM,C11-0000497-LIC,ZASP,Return,Standard,21188222,1A4060300008981000168707,transfer,1A4060300008981000165172,".2020-10-30_Kirkwood OG_P4., .2020-10-31_Kirkw...",Accepted,False,False,Edible (weight - each),FLAV-Apple-Belts-M-E-100,passed,NaN,30.0,Each,NaN,30.0,Each,40.0,Grams
2638,INCOMING_FROM_VENDOR,C11-0000497-LIC,0002949880,2022-01-13,2022-01-13 22:02:02+00:00,Standard,C10-0000223-LIC,L B GREEN ROOM,C11-0000497-LIC,ZASP,Return,Standard,21188231,1A4060300008981000168712,transfer,1A4060300008981000165181,".2020-10-30_Kirkwood OG_P4., .2020-10-31_Kirkw...",Accepted,False,False,Edible (weight - each),FLAV-Strawberry-Belts-M-E-100,passed,

In [157]:
df_in[df_in['package_id'].isin (s)][['package_id','shipment_type_name']].groupby('shipment_type_name').count()

,package_id
shipment_type_name,
Return,94
State Authorized,1
Transfer,25


In [158]:
df_in_ext_clean[df_in_ext_clean['received_datetime'] >= '2022-01-01'][['package_id','shipment_type_name']].groupby('shipment_type_name').count()

,package_id
shipment_type_name,
Return,100
State Authorized,1
Transfer,1693
Wholesale Manifest,11


In [164]:
df_out[df_out['package_id'] == '25131220']

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name
6519,OUTGOING_TO_PAYOR,C11-0000497-LIC,0003484890,2022-05-13,2022-05-17 00:50:09+00:00,C11-0000497-LIC,ZASP,C10-0001014-LIC,THIRD & KENMORE LLC,Wholesale Manifest,Wholesale,25131220,1A4060300008981000181595,transfer,1A4060300008981000179945,".2020-10-30_Kirkwood OG_P4., .2020-10-31_Kirkw...",Accepted,False,False,Edible (weight - each),FLAV-Blueberry-Belts-M-E-100,passed,172.5,30.0,Each,30.0,Each,172.5,40.0,Grams


# track packages

In [112]:
in_data = df_in_ext_clean[['received_datetime','shipper_facility_license_number','shipper_facility_name',\
                            'shipment_type_name','package_id','product_name','shipped_quantity','received_quantity','received_unit_of_measure']]
in_data.columns = ['received_datetime_in','shipper_facility_license_number_in','shipper_facility_name_in',\
                            'shipment_type_name_in','package_id','product_name_in','shipped_quantity_in','received_quantity_in','received_unit_of_measure_in']

In [118]:
out_data = df_out[['received_datetime',\
                      'recipient_facility_license_number','recipient_facility_name','package_id',\
                      'product_name','shipped_quantity','received_quantity','received_unit_of_measure']]
out_data.columns = ['received_datetime_out',\
                      'recipient_facility_license_number_out','recipient_facility_name_out','package_id',\
                      'product_name_out','shipped_quantity_out','received_quantity_out','received_unit_of_measure_out']


In [119]:
j = in_data.merge(out_data,how = 'left',on = ['package_id'])

In [120]:
j.shape

(21939, 16)

# criterias

1. received before send out
2. sender of package is not the receiver 

In [159]:
ms = j[(j['received_datetime_in'] < j['received_datetime_out'])\
       &(j['shipper_facility_license_number_in'] != j['recipient_facility_license_number_out'])]






In [162]:
ms[ms['received_datetime_in'] >= '2022-01-01']

,received_datetime_in,shipper_facility_license_number_in,shipper_facility_name_in,shipment_type_name_in,package_id,product_name_in,shipped_quantity_in,received_quantity_in,received_unit_of_measure_in,received_datetime_out,recipient_facility_license_number_out,recipient_facility_name_out,product_name_out,shipped_quantity_out,received_quantity_out,received_unit_of_measure_out
1031,2022-03-09 22:24:46+00:00,C11-0000995-LIC,ACCENTIAN INC.,Transfer,22914773,Potent Goods | .5g Disposable | Offline,760.0,523.0,Each,2022-03-22 17:35:50+00:00,C12-0000392-LIC,"Terp Wholesale, LLC",Potent Goods | .5g Disposable | Offline,228.0,NaN,None


In [161]:
ms

,received_datetime_in,shipper_facility_license_number_in,shipper_facility_name_in,shipment_type_name_in,package_id,product_name_in,shipped_quantity_in,received_quantity_in,received_unit_of_measure_in,received_datetime_out,recipient_facility_license_number_out,recipient_facility_name_out,product_name_out,shipped_quantity_out,received_quantity_out,received_unit_of_measure_out
1031,2022-03-09 22:24:46+00:00,C11-0000995-LIC,ACCENTIAN INC.,Transfer,22914773,Potent Goods | .5g Disposable | Offline,760.0,523.0,Each,2022-03-22 17:35:50+00:00,C12-0000392-LIC,"Terp Wholesale, LLC",Potent Goods | .5g Disposable | Offline,228.0,NaN,None
2194,2021-12-01 21:22:21+00:00,C11-0000886-LIC,"XTRACTA DISTRIBUTION, LLC",Transfer,19876619,Flav - Live Resin Pod - Fruit Bomb - 0.5G,1220.0,1220.0,Each,2021-12-09 00:00:57+00:00,C11-0000497-LIC,ZASP,Flav - Live Resin Pod - Fruit Bomb - 0.5G,1226.0,1226.0,Each
3336,2021-08-26 15:08:18+00:00,C10-0000674-LIC,MEDALLION WELLNESS INC.,Transfer,17285285,THC-GorillaGoo-H-PR-1.0,50.0,50.0,Each,2021-08-26 18:20:27+00:00,C10-0000577-LIC,"Medallion Wellness, Inc",THC-GorillaGoo-H-PR-1.0,50.0,50.0,Each
3338,2021-08-26 15:08:18+00:00,C10-0000674-LIC,MEDALLION WELLNESS INC.,Transfer,17285281,THC-24K-I-PR-1.0,25.0,25.0,Each,2021-08-26 18:20:27+00:00,C10-0000577-LIC,"Medallion Wellness, Inc",THC-24K-I-PR-1.0,25.0,25.0,Each
3341,2021-08-26 15:08:18+00:00,C10-0000674-LIC,MEDALLION WELLNESS INC.,Transfer,17285279,THC-PinkCookies-H-F-3.5,64.0,64.0,Each,2021-08-26 18:20:27+00:00,C10-0000577-LIC,"Medallion Wellness, Inc",THC-PinkCookies-H-F-3.5,64.0,64.0,Each
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21930,2020-01-04 01:49:29+00:00,C11-0000150-LIC,"NABIONE, INC.",Transfer,1832482,Madrone - Strawberry SFV - 3.5g Flower,112.0,112.0,Grams,2020-01-15 22:28:11+00:00,C10-0000362-LIC,"PERENNIAL HOLISTIC WELLNESS CENTER, INC.",Madrone - Strawberry SFV - 3.5g Flower,112.0,112.0,Grams
21931,2020-01-04 01:50:38+00:00,C11-0000150-LIC,"NABIONE, INC.",Transfer,1806965,Beezle - Lemon Love - 1g Live Resin Sugar,55.0,55.0,Each,2020-01-09 00:46:51+00:00,C10-0000360-LIC,HOLISTIC INC.,Beezle - Lemon Love - 1g Live Resin Sugar,55.0,55.0,Each
21932,2020-01-04 01:50:38+00:00,C11-0000150-LIC,"NABIONE, INC.",Transfer,1806958,Beezle - Crossroad Chem - 1g Cured Resin Budder,5.0,5.0,Each,2020-01-09 00:46:51+00:00,C10-0000360-LIC,HOLISTIC INC.,Beezle - Crossroad Chem - 1g Cured Resin Budder,5.0,5.0,Each
21933,2020-01-04 01:49:29+00:00,C11-0000150-LIC,"NABIONE, INC.",Transfer,1832483,Madrone - The Viper - 7g Flower,112.0,112.0,Grams,2020-01-15 22:28:11+00:00,C10-0000362-LIC,"PERENNIAL HOLISTIC WELLNESS CENTER, INC.",Madrone - The Viper - 7g Flower,112.0,112.0,Grams


In [ ]:
# need to lower case the names

# Vendor churn

In [ ]:
VC_START_DATE = '2020-01-01'
VC_END_DATE = '2022-05-01'
VC_MONTH_LIST = ['2022-02-01','2022-03-01','2022-04-01','2022-05-01']
VC_MONTH_END = VC_MONTH_LIST[-1]

In [ ]:
license_numbers

In [ ]:
df_in['delivery_type'].unique()

In [ ]:
df_out.head()

In [ ]:
test_pkg

In [ ]:
df_in[df_in['package_id'] == '24408766']

In [ ]:
df_out[df_out['package_id'] == '24408766']

In [ ]:
test_pkg[(test_pkg['recipient_facility_name'].notnull())&(test_pkg['received_datetime_x'] < test_pkg['received_datetime_y'])]


In [ ]:
df_in[df_in['package_id'] == '24408766']

In [ ]:
df_out[df_out['package_id'] == '24408766']

In [ ]:
df_out[df_out['recipient_facility_license_number'] == 'C12-0000392-LIC']

In [ ]:
test_pkg[test_pkg['recipient_facility_name'].notnull()].shape[0] / test_pkg.shape[0]

In [ ]:
churn,vc_matrix = client_surveillance_jupyter.calculate_vendor_churn(company_incoming_transfer_packages_dataframe,license_numbers,VC_START_DATE,VC_END_DATE,VC_MONTH_LIST)


In [ ]:
churn_short,vc_matrix_short = client_surveillance_jupyter.calculate_vendor_churn_short(company_incoming_transfer_packages_dataframe,license_numbers,VC_START_DATE,VC_END_DATE,VC_MONTH_LIST,VC_MONTH_END)


In [ ]:
churn_short

In [ ]:
ax = churn['%_inactive'].plot(marker = 'o',figsize = (12,8),label = 'VC old')
ax = churn_short['%_inactive'].plot(marker = 'o',figsize = (12,8),label = 'VC short')
vals = ax.get_yticks()
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals])
ax = plt.title('Rolling 4m vendor churn')
ax = plt.legend()

# Receiver wholesale price outlier check

In [ ]:
per_unit_incoming_package_sort,per_unit_incoming_product_sort = client_surveillance_jupyter.check_per_unit_incoming(company_incoming_transfer_packages_dataframe)


In [ ]:
# top 5
print('Top 5 per unit incoming packages')
per_unit_incoming_package_sort.head(20)

In [ ]:
print('Bottom 5 per unit incoming packages')
per_unit_incoming_package_sort.dropna().tail(5)


In [ ]:
# look at large cost items if any
company_incoming_transfer_packages_dataframe[company_incoming_transfer_packages_dataframe['per_unit_incoming'] > 10000]

# COGS analysis

In [ ]:
df_in = company_incoming_transfer_packages_dataframe[company_incoming_transfer_packages_dataframe['per_unit_incoming'] <= 10000]
df_inventory = company_inventory_packages_dataframe
df_sales = deduped_sales_receipts_with_transactions_dataframe

In [ ]:
df_summary_simp,df_cogs_average_product = client_surveillance_jupyter.cogs_analysis(df_in,df_sales,'monthly',company_state)


In [ ]:
df_summary_simp

In [ ]:
df_summary_simp_uom,df_cogs_average_product_uom = client_surveillance_jupyter.cogs_analysis_uom(df_in,df_sales,'monthly',company_state)


In [ ]:
j = df_sales.merge(df_in,how= 'left',left_on = 'tx_package_id',right_on = 'package_id')

In [ ]:
j[(j['tx_unit_of_measure']!= j['shipped_unit_of_measure'])&(j['shipped_unit_of_measure'].notnull())][['sales_month','tx_package_id','tx_product_name','tx_product_category_name','tx_unit_of_measure','shipped_unit_of_measure','per_unit','per_unit_incoming']].head()



In [ ]:
import seaborn as sns; sns.set()
sns.set(rc={'figure.figsize':(10,5)})

In [ ]:
df_summary_simp['margin_%'] - df_summary_simp_uom['margin_%']

In [ ]:
plt.scatter(df_summary_simp['margin_%'] ,df_summary_simp_uom['margin_%'] )
plt.xlabel('GM% original')
plt.ylabel('GM% improved')


In [ ]:
df_summary_simp[[ 'revenue', 'cogs', 'margin_$', 'margin_%']]

In [ ]:
df_summary_simp_uom[[ 'revenue', 'cogs', 'margin_$', 'margin_%']]

# Revenue change vs state

In [ ]:
if COMPANY_IDENTIFIER == ['NECC']:
    a = client_surveillance_jupyter.calculate_quarterly_sum_gmv_ma_short(df_summary_simp)
    a

In [ ]:
if COMPANY_IDENTIFIER == ['NECC']:
    round(a.dropna()['total'].sum(),2)

In [ ]:
a = client_surveillance_jupyter.get_gmv_change(company_state,df_summary_simp)
round(a.dropna()['total'].sum(),2)

In [ ]:
a

# Inventory

In [ ]:
today = date.today()
today

In [ ]:
COMPANY_IDENTIFIER

In [ ]:
license_numbers

In [ ]:
df_inventory.head()

In [ ]:
df_in['created_date'] = pd.to_datetime(df_in['created_date'])

In [ ]:
df_in_recent

In [ ]:
df_in_recent = df_in[df_in['created_date'] >= '2022-01-01']

In [ ]:
df_in_recent.head()

In [ ]:
df_inventory['package_id'].nunique()

In [ ]:
df_inventory_license = client_surveillance_jupyter.calculate_inventory_valuation(df_in_recent,df_inventory,license_numbers,today)

In [ ]:
df_inventory_license

In [ ]:
df_inventory_license_msrp = client_surveillance_jupyter.calculate_msrp_based_inventory_valuation(df_in,df_sales,df_inventory,license_numbers,today)

In [ ]:
df_inventory_license_msrp

In [ ]:
df_inventory_fresh,_ = client_surveillance_jupyter.calculate_inventory_valuation_fresh(df_in,df_inventory,license_numbers,today,0,90)

In [ ]:
df_inventory_fresh

In [ ]:
_['age_int'].describe()

# Quantity based inventory TO

In [ ]:
df_inventory.head()

In [ ]:
inventory_q = df_inventory[['package_id','unit_of_measure','quantity']].groupby(['package_id','unit_of_measure']).sum()



In [ ]:
inventory_q = inventory_q.reset_index()
inventory_q

In [ ]:
df_sales_3m = df_sales[df_sales['sales_month'] >= '2022-03']

In [ ]:
sales_q = df_sales_3m[['tx_package_id','tx_unit_of_measure','tx_quantity_sold']].groupby(['tx_package_id','tx_unit_of_measure']).sum()
sales_q = sales_q.reset_index()


In [ ]:
j = inventory_q.merge(sales_q,how = 'left',left_on = ['package_id','unit_of_measure'],right_on = ['tx_package_id','tx_unit_of_measure'])



In [ ]:
j['to'] = j['tx_quantity_sold'] / j['quantity'] * 4

In [ ]:
j[j['to'] >= 50]

In [ ]:
j['to'].describe()

# Repayment score

In [ ]:
company_repayment_data

In [ ]:
company_repayment_data_due = company_repayment_data[company_repayment_data['adjusted_maturity_date'] < today]

In [ ]:
company_repayment_data_due


In [ ]:
company_repayment_data_due['weight'] = company_repayment_data_due['loan_amount'] / sum(company_repayment_data_due['loan_amount'])

In [ ]:
len(company_repayment_data_due)

In [ ]:
if len(company_repayment_data_due) == 0:
    weighted_dpd = numpy.nan
else:
    weighted_dpd = sum(company_repayment_data_due['days_late'] * company_repayment_data_due['weight'])

    

In [ ]:
weighted_dpd

In [ ]:
repayment_score = client_surveillance_jupyter.get_short_repayment_score(weighted_dpd)
repayment_score

# Create Template: updated

In [ ]:
current_month = '2022-05'
full_score = 45

In [ ]:
license_numbers

In [ ]:
template_new_ = client_surveillance_jupyter.create_template_update(a,df_summary_simp,df_inventory_license,df_inventory_license_msrp,df_inventory_fresh,churn_short,retial_license_check_df,license_numbers,company_state,current_month)



In [ ]:
template_new_

# Output data 

In [ ]:
license_numbers

## others

In [ ]:
# for only 1 license
license_number = license_numbers[0]
template_new_.to_csv('../analysis/surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month + '/template/' + COMPANY_IDENTIFIER[0] + '_'+ str(today) + '_' +license_number +'_template.csv')
df_inventory_license.to_csv('../analysis/surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/INV/' + COMPANY_IDENTIFIER[0] + '_'+ str(today) + '_' +license_number +'_inventory_val.csv')
df_inventory_fresh.to_csv('../analysis/surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/INV/' + COMPANY_IDENTIFIER[0] + '_'+ str(today) + '_' +license_number +'_inventory_fresh.csv')
df_inventory_license_msrp.to_csv('../analysis/surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/INV/' + COMPANY_IDENTIFIER[0] + '_'+ str(today) + '_' +license_number +'_inventory_msrp.csv')
df_summary_simp.to_csv('surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/COGS/' + COMPANY_IDENTIFIER[0] + '_'+ VC_END_DATE + '_' +license_number +'_cogs.csv')
vc_matrix_short.to_csv('surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/VC/' + COMPANY_IDENTIFIER[0] + '_'+ VC_END_DATE + '_' +license_number +'_vc_matrix.csv')
churn_short.to_csv('surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/VC/' + COMPANY_IDENTIFIER[0] + '_'+ VC_END_DATE + '_' +license_number +'_vc.csv')
a.to_csv('surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/rev_change_vs_state/' + COMPANY_IDENTIFIER[0] + '_'+ VC_END_DATE + '_' +license_number +'_rev_change_vs_state.csv')
#retial_license_check_df



In [ ]:
# multiple licenses
template_new_.to_csv('../analysis/surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month + '/template/' + COMPANY_IDENTIFIER[0] + '_'+ str(today) +'_template.csv')
df_inventory_license.to_csv('../analysis/surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/INV/' + COMPANY_IDENTIFIER[0] + '_'+ str(today)  +'_inventory_val.csv')
df_inventory_fresh.to_csv('../analysis/surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/INV/' + COMPANY_IDENTIFIER[0] + '_'+ str(today) +'_inventory_fresh.csv')
df_inventory_license_msrp.to_csv('../analysis/surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/INV/' + COMPANY_IDENTIFIER[0] + '_'+ str(today) +'_inventory_msrp.csv')
df_summary_simp.to_csv('surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/COGS/' + COMPANY_IDENTIFIER[0] + '_'+ VC_END_DATE +'_cogs.csv')
vc_matrix_short.to_csv('surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/VC/' + COMPANY_IDENTIFIER[0] + '_'+ VC_END_DATE  +'_vc_matrix.csv')
churn_short.to_csv('surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/VC/' + COMPANY_IDENTIFIER[0] + '_'+ VC_END_DATE  +'_vc.csv')
a.to_csv('surveillance/'+COMPANY_IDENTIFIER[0]+'/'+current_month +'/rev_change_vs_state/' + COMPANY_IDENTIFIER[0] + '_'+ VC_END_DATE  +'_rev_change_vs_state.csv')
#retial_license_check_df



# Appendix

In [ ]:
# def calculate_quarterly_sum_gmv_ma_short(cogs_analysis_df):

#     gmv_df = cogs_analysis_df[(cogs_analysis_df['date'] >= '2021-09')&(cogs_analysis_df['date'] <= '2022-02')][['revenue_change']]
#     gmv_df['sum_gmv_change_ma'] = [-0.07,0.09,-0.11,0.02]
#     gmv_df['weight'] = [0.76,0.84,0.92,1]
#     gmv_df['variance'] = gmv_df['revenue_change'] - gmv_df['sum_gmv_change_ma']
#     gmv_df['points'] = [gmv_change_variance_point_mapping(n) for n in gmv_df['variance']]
#     gmv_df.replace([numpy.inf, -numpy.inf], numpy.nan, inplace=True)
#     gmv_df['total'] = gmv_df['points'] * gmv_df['weight']
#     return gmv_df
    